In [1]:
import numpy as np
import torch.nn.functional as F
from transform_factory import center_crop_224
from torchvision import transforms
import torch
from tqdm import tqdm

In [2]:
with open("results/train_seed_0_pred_orig_eval_orig_transform_both_sign_all_reduction_sum/n01440764_752.JPEG_expl_GradCAM_sample_2000_sigma_0.0_seed_0_orig_true_config.npy", "rb") as f:
    orig_expl = np.load(f, allow_pickle=True)
    true_expls = np.load(f, allow_pickle=True)
    configs = np.load(f, allow_pickle=True)

print(orig_expl.shape, true_expls.shape, configs.shape)
orig_expl = F.interpolate(torch.tensor(orig_expl).unsqueeze(0), (224, 224), mode='bilinear').squeeze(0).numpy()
print(orig_expl.shape, orig_expl)

(1, 11, 11) (2000, 1, 11, 11) (2000,)
(1, 224, 224) [[[-0.00058214 -0.00058214 -0.00058214 ...  0.00041289  0.00041289
    0.00041289]
  [-0.00058214 -0.00058214 -0.00058214 ...  0.00041289  0.00041289
    0.00041289]
  [-0.00058214 -0.00058214 -0.00058214 ...  0.00041289  0.00041289
    0.00041289]
  ...
  [-0.00096525 -0.00096525 -0.00096525 ... -0.00338584 -0.00338584
   -0.00338584]
  [-0.00096525 -0.00096525 -0.00096525 ... -0.00338584 -0.00338584
   -0.00338584]
  [-0.00096525 -0.00096525 -0.00096525 ... -0.00338584 -0.00338584
   -0.00338584]]]


In [4]:
indicies = np.arange(0, 2000, 1)
np.random.shuffle(indicies)

cal_idx, val_idx = indicies[:1000], indicies[1000:]

scores = []

for true_expl, config in tqdm(zip(true_expls[cal_idx], configs[cal_idx])):

    T_inv_spatial = transforms.Compose([
        transforms.RandomRotation((-config['rot_angle'], -config['rot_angle'])),
        transforms.RandomVerticalFlip(config['flip_vertical']),
        transforms.RandomHorizontalFlip(config['flip_horizon']),
        
    ])
    
    true_expl = center_crop_224(T_inv_spatial(F.interpolate(torch.tensor(true_expl).unsqueeze(0), (322, 322), mode='bilinear'))).squeeze(0).numpy()

    # pp_true_expls.append(true_expl)
    scores.append(np.abs(true_expl - orig_expl))


scores = np.stack(scores)
print(scores.shape)
    

1000it [02:17,  7.28it/s]

(1000, 1, 224, 224)


In [5]:
test_expls =[]
for true_expl, config in tqdm(zip(true_expls[val_idx], configs[val_idx])):

    T_inv_spatial = transforms.Compose([
        transforms.RandomRotation((-config['rot_angle'], -config['rot_angle'])),
        transforms.RandomVerticalFlip(config['flip_vertical']),
        transforms.RandomHorizontalFlip(config['flip_horizon']),
        
    ])
    
    test_expls.append(center_crop_224(T_inv_spatial(F.interpolate(torch.tensor(true_expl).unsqueeze(0), (322, 322), mode='bilinear'))).squeeze(0).numpy())

test_expls = np.stack(test_expls)



1000it [00:52, 19.07it/s]


In [6]:
def qhat(score, alpha:float):
    n = score.shape[0]
    q_hat = np.quantile(score, np.ceil((n+1) * (1-alpha)) / n, axis = 0)

    return q_hat

def get_conf_interval(expl: np.ndarray, q_hat: np.ndarray):
    high = expl + q_hat
    low = expl - q_hat
    return (low, high)

def calc_coverage_prob(true: np.ndarray, conf_low: np.ndarray, conf_high: np.ndarray):
    is_cover = np.logical_and(conf_low <= true, true <= conf_high)
    coverage_prob = np.sum(is_cover, axis = 0) / true.shape[0]

    return coverage_prob

In [9]:
results = []

for alpha in np.arange(0.05, 1, 0.05):
    q_hat = qhat(scores, alpha)

    conf_low, conf_high = get_conf_interval(orig_expl, q_hat)
    
    coverage_prob = calc_coverage_prob(test_expls, conf_low, conf_high)

    zc_rate = zero_contain_rate(conf_high, conf_low)

    # print(np.mean(coverage_prob))
    results.append({
        'img': img,
        'expl_method': expl_method,
        'alpha': alpha,
        'coverage_prob': coverage_prob,
        'zero_contain_rate': zc_rate,
        'conf_high': conf_high,
        'conf_low': conf_low
    })



(1, 224, 224)
(1, 224, 224)
(1, 224, 224)
(1, 224, 224)


KeyboardInterrupt: 